In [9]:
import cv2
import numpy as np
from scipy import ndimage
import math
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.models import model_from_json
import copy

# Sudoku solver

In [10]:
# Class to keep data about the "Best" cell
class EntryData:
    def __init__(self, r, c, n):
        self.row = r
        self.col = c
        self.choices = n

    def set_data(self, r, c, n):
        self.row = r
        self.col = c
        self.choices = n

## Solving Sudoku using Best-first Search

In [11]:
# Solve Sudoku using Best-first search
def solve_sudoku(matrix):
    cont = [True]
    # See if it is even possible to have a solution
    for i in range(9):
        for j in range(9):
            if not can_be_correct(matrix, i, j): # If it is not possible, stop
                return
    sudoku_helper(matrix, cont) # Otherwise try to solve the Sudoku puzzle
    
# Helper functions

def sudoku_helper(matrix, cont):
    if not cont[0]: # Stopping point 1
        return

    # Find the best entry (The one with the least possibilities)
    best_candidate = EntryData(-1, -1, 100)
    for i in range(9):
        for j in range(9):
            if matrix[i][j] == 0: # If it is unfilled
                num_choices = count_choices(matrix, i, j)
                if best_candidate.choices > num_choices:
                    best_candidate.set_data(i, j, num_choices)

    # If there were not any choices
    if best_candidate.choices == 100: 
        cont[0] = False
        return

    row = best_candidate.row
    col = best_candidate.col

    # If found the best candidate, try to fill 1-9
    for j in range(1, 10):
        if not cont[0]:
            return
        matrix[row][col] = j
        if can_be_correct(matrix, row, col):
            sudoku_helper(matrix, cont)

    if not cont[0]:
        return
    matrix[row][col] = 0 # Backtracking

# Count the number of choices haven't been considered
def count_choices(matrix, i, j):
    can_pick = [True,True,True,True,True,True,True,True,True,True]; # From 0 to 9 - drop 0

    # Check row
    for k in range(9):
        can_pick[matrix[i][k]] = False

    # Check col
    for k in range(9):
        can_pick[matrix[k][j]] = False

    # Check 3x3 square
    r = i // 3
    c = j // 3
    for row in range(r*3, r*3+3):
        for col in range(c*3, c*3+3):
            can_pick[matrix[row][col]] = False

    # Count
    count = 0
    for k in range(1, 10):  # 1 to 9
        if can_pick[k]:
            count += 1

    return count

# Return true if the current cell doesn't create any violation
def can_be_correct(matrix, row, col):

    # Check row
    for c in range(9):
        if matrix[row][col] != 0 and col != c and matrix[row][col] == matrix[row][c]:
            return False

    # Check column
    for r in range(9):
        if matrix[row][col] != 0 and row != r and matrix[row][col] == matrix[r][col]:
            return False

    # Check 3x3 square
    r = row // 3
    c = col // 3
    for i in range(r*3, r*3+3):
        for j in range(c*3, c*3+3):
            if row != i and col != j and matrix[i][j] != 0 and matrix[i][j] == matrix[row][col]:
                return False

    return True

In [12]:
# Return true if the whole board has been occupied by some non-zero number, which means the current board is the solution
def all_board_non_zero(matrix):
    for i in range(9):
        for j in range(9):
            if matrix[i][j] == 0:
                return False
    return True

# Real time Solver

## Display Solution Board

In [13]:
import pygame

WIDTH = 550
background_color = (251, 247, 245)
original_grid_element_color = (52, 31, 151)
solution_grid_color = (255, 0, 0)

def displaySolution(grid, user_grid):
    pygame.init()
    win = pygame.display.set_mode((WIDTH, WIDTH))
    pygame.display.set_caption("Sudoku")
    win.fill(background_color)
    myfont = pygame.font.SysFont('Comic Sans MS', 35)

    for i in range(0,10):
        if(i%3 == 0):
            pygame.draw.line(win, (0,0,0), (50 + 50*i, 50), (50 + 50*i ,500 ), 4 )
            pygame.draw.line(win, (0,0,0), (50, 50 + 50*i), (500, 50 + 50*i), 4 )

        pygame.draw.line(win, (0,0,0), (50 + 50*i, 50), (50 + 50*i ,500 ), 2 )
        pygame.draw.line(win, (0,0,0), (50, 50 + 50*i), (500, 50 + 50*i), 2 )
    pygame.display.update()

    for i in range(0, len(grid[0])):
        for j in range(0, len(grid[0])):
            # if(0<grid[i][j]<10):
            if(user_grid[i][j] != 0):
                value = myfont.render(str(grid[i][j]), True, original_grid_element_color)
            elif (user_grid[i][j] == 0):
                value = myfont.render(str(grid[i][j]), True, solution_grid_color)
            win.blit(value, ((j+1)*50 + 15, (i+1)*50 ))
    pygame.display.update()

    while True:
        pygame.init()
        events = pygame.event.get()
        for event in events:
            if event.type == pygame.QUIT:
                pygame.quit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_r:
                    pygame.quit()

## Real time recognizing and sudoku solving
With the help of helper functions above, we do the following steps:
1. Detect the Sudoku board 
2. Extract and recognize ditgits in the board 
3. Apply the algorithm to solve the Sudoku board detected
4. Print the result in a new pygame window for user-friendly purpose

### Helper functions for Real time Sudoku solver

In [14]:
# Compare every single elements of 2 matrices and return if all corresponding entries are equal 
# This function helps us to know if there is new board to be detected in the frame
def two_matrices_are_equal(matrix_1, matrix_2, row, col):
    for i in range(row):
        for j in range(col):
            if matrix_1[i][j] != matrix_2[i][j]:
                return False
    return True

# The first criteria for detecting
# We need to check if the contour is a Sudoku board or not: Length of Sides cannot be too different (sudoku board we use is a 9x9 square)
# Return if the longest size is > the shortest size * eps_scale
def side_lengths_are_too_different(A, B, C, D, eps_scale):
    AB = math.sqrt((A[0]-B[0])**2 + (A[1]-B[1])**2)
    AD = math.sqrt((A[0]-D[0])**2 + (A[1]-D[1])**2)
    BC = math.sqrt((B[0]-C[0])**2 + (B[1]-C[1])**2)
    CD = math.sqrt((C[0]-D[0])**2 + (C[1]-D[1])**2)
    shortest = min(AB, AD, BC, CD)
    longest = max(AB, AD, BC, CD)
    return longest > eps_scale * shortest

# The second criteria for detecting
# We need to check if the contour is a Sudoku board or not: All 4 angles has to be approximately 90 degree
# Approximately 90 degress with tolerance epsilon
def approx_90_degrees(angle, epsilon):
    return abs(angle - 90) < epsilon

# The Sudoku board will be chopped into 9x9 small square image, each of those image is a "crop_image"
# This function is used for seperating the digit from noise in "crop_image" (such as black edges)
def largest_connected_component(image):
    
    image = image.astype('uint8')
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(image, connectivity=8)
    sizes = stats[:, -1]

    if(len(sizes) <= 1):
        blank_image = np.zeros(image.shape)
        blank_image.fill(255)
        return blank_image

    max_label = 1
    # Start from component 1 (not 0) because we want to leave out the background
    max_size = sizes[1]

    for i in range(2, nb_components):
        if sizes[i] > max_size:
            max_label = i
            max_size = sizes[i]

    img2 = np.zeros(output.shape)
    img2.fill(255)
    img2[output == max_label] = 0
    return img2

# Using the equation to calculate cosin, this function returns the angle between 2 vectors in degrees
def angle_between(vector_1, vector_2):
    unit_vector_1 = vector_1 / np.linalg.norm(vector_1)
    unit_vector2 = vector_2 / np.linalg.norm(vector_2)
    dot_droduct = np.dot(unit_vector_1, unit_vector2)
    angle = np.arccos(dot_droduct)
    return angle * 57.2958  # Convert to degree

# Calculate how to centralize the image using its center of mass to detect the board even if the Sudoku board is not properly showed
def get_best_shift(img):
    cy, cx = ndimage.measurements.center_of_mass(img)
    rows, cols = img.shape
    shiftx = np.round(cols/2.0-cx).astype(int)
    shifty = np.round(rows/2.0-cy).astype(int)
    return shiftx, shifty

# Apply Affine transform to shift the image using what the get_best_shift function above returns
def shift(img,sx,sy):
    rows,cols = img.shape
    M = np.float32([[1,0,sx],[0,1,sy]])
    shifted = cv2.warpAffine(img,M,(cols,rows))
    return shifted

# Get 4 corners from contour - the corners of the Sudoku board detected
def get_corners_from_contours(contours, corner_amount=4, max_iter=200):

    coefficient = 1
    while max_iter > 0 and coefficient >= 0:
        max_iter = max_iter - 1

        epsilon = coefficient * cv2.arcLength(contours, True)

        poly_approx = cv2.approxPolyDP(contours, epsilon, True)
        hull = cv2.convexHull(poly_approx)
        if len(hull) == corner_amount:
            return hull
        else:
            if len(hull) > corner_amount:
                coefficient += .01
            else:
                coefficient -= .01
    return None

# Preprocess the image for digit recognition
def preprocess(img_array):
    new_array = img_array.reshape(-1, 28, 28, 1)
    new_array = new_array.astype('float32')
    new_array /= 255
    return new_array

# Show the image
def showImage(img, name, width, height):
    new_image = np.copy(img)
    new_image = cv2.resize(new_image, (width, height))
    cv2.imshow(name, new_image)

### Primary function for Real time solver

In [15]:
# The primary function for Real time Sudoku solver
def recognize_and_solve_sudoku(image, model, old_sudoku):

    clone_image = np.copy(image)    # deep copy to use later

    # Convert to a gray image, blur that gray image for easier detection
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Showing gray image:
    # cv2.imshow("gray", gray) 
    
    Gblur = cv2.GaussianBlur(gray, (5,5), 0)
    # Showing blurred image:
    # cv2.imshow("blur", Gblur)
    
    # Apply adaptiveThreshold
    aThresh = cv2.adaptiveThreshold(Gblur, 255, 1, 1, 11, 2)
    # Showing the image after applying threshold:
    # cv2.imshow('thres', aThresh)

    # Find all contours
    contours, _ = cv2.findContours(aThresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # Showing the contours:
    # cv2.drawContours(image, contours, -1, (0,255,0), 3)
    # cv2.imshow("contours", image)

    # Assuming the Sudoku board is the biggest contour, we need to extract it
    max_area = 0
    biggest_contour = None
    for c in contours:
        area = cv2.contourArea(c)
        if area > max_area:
            max_area = area
            biggest_contour = c

    if biggest_contour is None:    # No sudoku   
        return image

    # Get 4 corners of the biggest contour
    corners = get_corners_from_contours(biggest_contour, 4)

    if corners is None:        # No sudoku
        return image

    # Having 4 corners, we need to locate the top left - top right - bottom left - bottom right corners
    rect = np.zeros((4, 2), dtype = "float32")
    corners = corners.reshape(4,2)

    # Find top left (Knowing that sum of its coordinates is the smallest)
    sum = 10000
    index = 0
    for i in range(4):
        if(corners[i][0]+corners[i][1] < sum):
            sum = corners[i][0]+corners[i][1]
            index = i
    rect[0] = corners[index]
    corners = np.delete(corners, index, 0)

    # Find bottom right (Knowing that sum of its coordinates is the largest)
    sum = 0
    for i in range(3):
        if(corners[i][0]+corners[i][1] > sum):
            sum = corners[i][0]+corners[i][1]
            index = i
    rect[2] = corners[index]
    corners = np.delete(corners, index, 0)

    # Find top right and bottom left (Comparing 2 remaining points)
    if(corners[0][0] > corners[1][0]):
        rect[1] = corners[0]
        rect[3] = corners[1]

    else:
        rect[1] = corners[1]
        rect[3] = corners[0]

    rect = rect.reshape(4,2)


    # After having found 4 corners of the Sudoku board, we use ABCD as their notations
    A = rect[0]         # Top left corner
    B = rect[1]         # Top right corner
    C = rect[2]         # Bottom left corner
    D = rect[3]         # Bottom right corner

    # Checking the first criteria aformentioned: The Lengths of AB, AD, BC, DC have to be approximately equal
    eps_scale = 1.2     # Longest cannot be longer than eps_scale * shortest
    if(side_lengths_are_too_different(A, B, C, D, eps_scale)):
        return image

    # Checking the second criteria aformentioned: All 4 angles are not approximately 90 degrees (with tolerance = epsAngle)
    AB = B - A
    AD = D - A
    BC = C - B
    DC = C - D
    eps_angle = 20
    if not (approx_90_degrees(angle_between(AB,AD), eps_angle) and approx_90_degrees(angle_between(AB,BC), eps_angle)
    and approx_90_degrees(angle_between(BC,DC), eps_angle) and approx_90_degrees(angle_between(AD,DC), eps_angle)):
        return image

    # Calculating the width of our Sudoku board
    (tl, tr, br, bl) = rect
    width_A = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    width_B = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))

    # Calculating the height of our Sudoku board
    height_A = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    height_B = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))

    # Calculate final dimensions
    max_width = max(int(width_A), int(width_B))
    max_height = max(int(height_A), int(height_B))

    # Construct our destination points which will be used to map the screen to a top-down view
    dst = np.array([
	    [0, 0],
	    [max_width - 1, 0],
	    [max_width - 1, max_height - 1],
	    [0, max_height - 1]], dtype = "float32")

    # Apply perspective transform and warp the perspective to grab the screen
    perspective_transformed_matrix = cv2.getPerspectiveTransform(rect, dst)
    warp = cv2.warpPerspective(image, perspective_transformed_matrix, (max_width, max_height))
    orginal_warp = np.copy(warp)

    # At this point, warp contains the chopped Sudoku board
    # Image processing to get ready for recognizing digits
    warp = cv2.cvtColor(warp, cv2.COLOR_BGR2GRAY)
    warp = cv2.GaussianBlur(warp, (5,5), 0)
    warp = cv2.adaptiveThreshold(warp, 255, 1, 1, 11, 2)
    warp = cv2.bitwise_not(warp)
    _, warp = cv2.threshold(warp, 150, 255, cv2.THRESH_BINARY)

    # Create a grid to store Sudoku Board digits
    SIZE = 9
    grid = []
    for i in range(SIZE):
        row = []
        for j in range(SIZE):
            row.append(0)
        grid.append(row)

    height = warp.shape[0] // 9
    width = warp.shape[1] // 9

    offset_width = math.floor(width / 10)    # Offset is used to get rid of the boundaries
    offset_height = math.floor(height / 10)
    
    # Divide the Sudoku board into 9x9 square:
    for i in range(SIZE):
        for j in range(SIZE):
            
            # Crop with offset to exclude the boundaries
            crop_image = warp[height*i+offset_height:height*(i+1)-offset_height, width*j+offset_width:width*(j+1)-offset_width]

            # Remove all black lines near the edges
            ratio = 0.6
            # Top
            while np.sum(crop_image[0]) <= (1-ratio) * crop_image.shape[1] * 255:
                crop_image = crop_image[1:]
            # Bottom
            while np.sum(crop_image[:,-1]) <= (1-ratio) * crop_image.shape[1] * 255:
                crop_image = np.delete(crop_image, -1, 1)
            # Left
            while np.sum(crop_image[:,0]) <= (1-ratio) * crop_image.shape[0] * 255:
                crop_image = np.delete(crop_image, 0, 1)
            # Right
            while np.sum(crop_image[-1]) <= (1-ratio) * crop_image.shape[0] * 255:
                crop_image = crop_image[:-1]

            # Take the largestConnectedComponent (The digit) and remove all noises
            crop_image = cv2.bitwise_not(crop_image)
            crop_image = largest_connected_component(crop_image)

            # Resize
            digit_pic_size = 28
            crop_image = cv2.resize(crop_image, (digit_pic_size,digit_pic_size))

            # If this is a white cell, set grid[i][j] to 0 and continue on the next image:

            # Criteria 1 for detecting white cell: Having too little black pixels
            if crop_image.sum() >= digit_pic_size**2*255 - digit_pic_size * 1 * 255:
                grid[i][j] == 0
                continue    # Move on if we have a white cell

            # Criteria 2 for detecting white cell: Huge white area in the center
            center_width = crop_image.shape[1] // 2
            center_height = crop_image.shape[0] // 2
            x_start = center_height // 2
            x_end = center_height // 2 + center_height
            y_start = center_width // 2
            y_end = center_width // 2 + center_width
            center_region = crop_image[x_start:x_end, y_start:y_end]

            if center_region.sum() >= center_width * center_height * 255 - 255:
                grid[i][j] = 0
                continue    # Move on if we have a white cell

            # Store the number of rows and cols
            rows, cols = crop_image.shape

            # Apply Binary Threshold to make digits clearer
            _, crop_image = cv2.threshold(crop_image, 200, 255, cv2.THRESH_BINARY)
            crop_image = crop_image.astype(np.uint8)

            # Centralize the image according to its center of mass
            crop_image = cv2.bitwise_not(crop_image)
            shift_x, shift_y = get_best_shift(crop_image)
            shifted = shift(crop_image,shift_x,shift_y)
            crop_image = shifted

            crop_image = cv2.bitwise_not(crop_image)

            # Convert to proper format to recognize
            crop_image = preprocess(crop_image)

            # Recognize digits
            prediction = model.predict([crop_image])
            grid[i][j] = np.argmax(prediction[0]) + 1

    user_grid = copy.deepcopy(grid)

    # Solve sudoku after we have recognizing each digits of the Sudoku board:

    # If this is the same board as last camera frame: No need to solve again
    if (not old_sudoku is None) and two_matrices_are_equal(old_sudoku, grid, 9, 9):
        if(all_board_non_zero(grid)):
            displaySolution(grid, user_grid)
    # If this is a different board
    else:
        solve_sudoku(grid) # Solve it
        if(all_board_non_zero(grid)): # If we got a solution
            displaySolution(grid, user_grid)
            old_sudoku = copy.deepcopy(grid)      # Keep the old solution

    # Apply inverse perspective transform and paste the solutions on top of the orginal image
    result_sudoku = cv2.warpPerspective(orginal_warp, perspective_transformed_matrix, (image.shape[1], image.shape[0])
                                        , flags=cv2.WARP_INVERSE_MAP)
    result = np.where(result_sudoku.sum(axis=-1,keepdims=True)!=0, result_sudoku, image)

    return result

# Load model and start Real time Sudoku Solver

In [ ]:
def showImage(img, name, width, height):
    new_image = np.copy(img)
    new_image = cv2.resize(new_image, (width, height))
    cv2.imshow(name, new_image)

# Load and set up Camera
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

# Set up a HD-resolution Camera
cap.set(3, 1280)    
cap.set(4, 720)

# Build the model
input_shape = (28, 28, 1)
num_classes = 9
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Load weights from pre-trained model
model.load_weights("digitRecognition.h5")

# Turn on the webcam and start Sudoku solver
old_sudoku = None
while(True):
    ret, frame = cap.read() # Read the frame
    if ret == True:
        sudoku_frame = recognize_and_solve_sudoku(frame, model, old_sudoku)
        showImage(sudoku_frame, "Real Time Sudoku Solver", 1066, 600)
        if cv2.waitKey(1) & 0xFF == ord('q'):  
                break
    else:
        break

cap.release()
cv2.destroyAllWindows()